In [4]:
search_for = 123
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0024302005767822266
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 350465005
type: [1, 1, 1, 1, 123] 123
cases of this type: 228886641
100000 15.97475951115811
200000 26.571934544352516
300000 27.10995264069919
400000 26.597481788999456
500000 30.743938664422956
600000 26.263356273633065
700000 25.89644309868738
800000 26.232052779009614
900000 25.05141249587079
1000000 11.523035077760964
1100000 25.610915553923874
1200000 25.89940417237355
1300000 25.53543889560612
1400000 26.070512421111722
1500000 26.422509848609646
1600000 24.820357765028536
1700000 25.736469825837123
1800000 25.286431148753977
1900000 10.28711962264377
2000000 26.0472161832464
2100000 30.13808901079231
2200000 29.646467930241496
2300000 29.58689186162341
2400000 30.584914970425032
2500000 28.536191894796616
2600000 28.542340120575236
2700000 27.775333220146784
2800000 18.172809178036513
2900000 22.701134718080453
3000000 

29400000 23.76990607619763
29500000 24.073420353012363
29600000 12.216135228807929
29700000 23.799269650486497
29800000 24.14578057007262
29900000 24.433843360278768
30000000 21.832562511737052
30100000 12.869707244683337
30200000 24.35196567257878
30300000 23.567245665563163
30400000 23.05341425563416
30500000 22.581093550442393
30600000 20.96976329893045
30700000 16.010250607226816
30800000 15.124535878963945
30900000 21.48376992267415
31000000 22.16832376396805
31100000 22.983622374147075
31200000 23.2569836255837
31300000 24.119288820901943
31400000 15.144613351059169
31500000 20.893306852943006
31600000 23.539552361941695
31700000 24.32270250682553
31800000 23.730161795000384
31900000 17.529007978014874
32000000 17.940225205724857
32100000 23.979478661710903
32200000 22.48638482097617
32300000 23.033855652730377
32400000 21.293427500291834
32500000 20.559528823878736
32600000 8.665572478893536
32700000 21.88465331569003
32800000 20.492123385597328
32900000 22.380268425067566
33000

59100000 11.143386731948128
59200000 19.737759826576355
59300000 20.17014826088863
59400000 20.94830093861583
59500000 20.959947419248284
59600000 22.307860101469387
59700000 21.719667507071424
59800000 21.285109086854906
59900000 20.597655416908296
60000000 21.375871371862115
60100000 9.31445129230551
60200000 18.976269120580476
60300000 18.82397563820573
60400000 19.197055489773007
60500000 9.640015037963583
60600000 17.312578216200954
60700000 19.400308635681657
60800000 18.84062079335361
60900000 16.717469853914384
61000000 13.550020545299141
61100000 19.76322380626012
61200000 20.801189953321263
61300000 20.88805670523763
61400000 21.082134592026048
61500000 21.72138299301363
61600000 21.383140110785853
61700000 20.823361222456416
61800000 19.824211683722083
61900000 16.98964224734985
62000000 12.149757350633662
62100000 18.232710797296356
62200000 19.632956426043002
62300000 19.10576753764977
62400000 7.63742102262393
62500000 19.067085754346817
62600000 18.386737868639056
627000

88800000 17.03577825560407
88900000 17.05163193606937
89000000 16.98110979393521
89100000 17.206749361360323
89200000 18.337685422842476
89300000 18.959297303610104
89400000 18.998193385262862
89500000 18.884667736474395
89600000 17.762681662823347
89700000 17.073365821307398
89800000 16.980569754762907
89900000 17.78053146735327
90000000 16.57936814682268
90100000 8.326184191450507
90200000 16.39708508437566
90300000 6.529360597097867
90400000 17.45321344490238
90500000 8.817119761087156
90600000 17.154377123146457
90700000 17.31420616053932
90800000 17.236555494965447
90900000 16.97929892207018
91000000 17.52817174401784
91100000 18.851864991788094
91200000 18.785657947626216
91300000 19.55590751492249
91400000 18.217608891303552
91500000 16.963375212230602
91600000 16.89648730865303
91700000 17.205048299044886
91800000 17.158943433741175
91900000 16.203324743300733
92000000 8.488597131491641
92100000 13.824596974314545
92200000 10.388512918292037
92300000 11.81661785618053
92400000 

117900000 2.344800124525454
118000000 2.2341583387709854
118100000 1.7588477734503083
118200000 1.7390681608903475
118300000 1.7640280172488592
118400000 1.7612033678075008
118500000 1.751018358051949
118600000 1.7319245086895922
118700000 1.7365600552482472
118800000 1.7334050645232597
118900000 1.7291033351018728
119000000 1.7346796704434322
119100000 2.553720977303919
119200000 23.864879031849842
119300000 21.324961172222316
119400000 22.58486526180703
119500000 20.636456818796663
119600000 21.86483528001363
119700000 20.892122427268315
119800000 20.485991264670254
119900000 21.635671512315877
120000000 10.81251511195273
120100000 2.6742278493592546
120200000 19.488107691559986
120300000 20.722853204601176
120400000 21.65487270341323
120500000 19.985051329827236
120600000 22.57988050435953
120700000 20.48478083762884
120800000 22.694172718026586
120900000 21.551700500100722
121000000 22.330158894049553
121100000 17.60286069094695
121200000 18.453130230053034
121300000 16.21867824709

...

233100000 2.1529978647139667
233200000 2.2129747074283364
233300000 2.057632724296669
233400000 2.133956008916381
233500000 1.1972139390287397
233600000 0.8345632665784491
233700000 1.5397790177950892
233800000 2.0991106094451943
233900000 1.9845895979828339
234000000 2.0831767146058877
234100000 2.0630328908293247
234200000 2.0503113125551806
234300000 2.0731408634693755
234400000 2.001324470272531
234500000 2.4140187414728764
234600000 3.20269723449965
234700000 1.892203648108393
234800000 2.829409823002239
234900000 2.085321865718901
235000000 2.4804184826503994
235100000 2.084742643349886
235200000 2.0358421211181117
235300000 2.368926230683674
235400000 0.5378590468535721
235500000 0.5245586469655202
235600000 2.231763860214045
235700000 1.918322528928982
235800000 2.1148521556063593
235900000 2.270720895733297
236000000 2.0294082464233405
236100000 2.43756235858098
236200000 2.073389400256028
236300000 2.1653190314192505
236400000 2.4115632376484175
236500000 2.1946841525829233
2

261800000 3.2720683217272097
261900000 3.1139908973561226
262000000 3.0815219169769588
262100000 3.145414484619141
262200000 3.023328696926713
262300000 3.14137018690299
262400000 3.8447995735617475
262500000 3.7526664527946214
262600000 3.406764430648198
262700000 3.0752090289569827
262800000 3.1801662086064653
262900000 3.258262423957974
263000000 3.2123143163842958
263100000 3.0040561408989697
263200000 3.4566004075728456
263300000 0.7345531247211164
263400000 0.7402278177195456
263500000 2.7639970592343808
263600000 3.229575178135693
263700000 3.2037579267263046
263800000 3.3889993287753755
263900000 3.2002426039499907
264000000 2.9148631523272326
264100000 3.229112690782219
264200000 3.3337901813897157
264300000 4.3656765265657045
264400000 4.048199239047975
264500000 3.637105417419347
264600000 3.378949306485878
264700000 3.724328732011259
264800000 3.7424928470120435
264900000 3.428745979101893
265000000 3.1344802421937388
265100000 3.1330741391953527
265200000 0.723329233309841

290600000 2.151890002430154
290700000 1.9630964385807215
290800000 2.2501269623081983
290900000 2.064318234207415
291000000 2.043342895209938
291100000 2.0908659743474423
291200000 0.7892039416474766
291300000 0.49637304475218386
291400000 1.8543265210302968
291500000 1.912162112622761
291600000 2.090283399644736
291700000 2.1281364781851173
291800000 2.2034907301152877
291900000 2.000345260738684
292000000 2.1323379090525507
292100000 2.1824545987408985
292200000 2.5638521011893878
292300000 2.1522414235594076
292400000 2.163997419127481
292500000 1.84079956348602
292600000 2.045609181747943
292700000 2.063441917380952
292800000 2.189960206497964
292900000 1.9422094107429384
293000000 1.694659514982975
293100000 0.48194930777538814
293200000 0.47960827622874574
293300000 2.3171626344497005
293400000 1.9756569650213223
293500000 2.006923840104729
293600000 2.0195541614018615
293700000 2.1467030973709758
293800000 1.8764237861234843
293900000 2.1711441262849305
294000000 2.2038208567371

318300000 1.3149401457454233
318400000 1.2459623139523468
318500000 1.20053481858425
318600000 1.371632861912085
318700000 1.2639737644228337
318800000 1.244033418416573
318900000 1.186904227402379
319000000 1.1479238902655542
319100000 1.1494193974218303
319200000 1.422327134776655
319300000 1.204517972568959
319400000 1.1406228699825043
319500000 1.2174267977028985
319600000 1.2407951651912332
319700000 1.1549912097721664
319800000 1.3557918540682858
319900000 1.1257312413802743
320000000 1.1685455578039057
320100000 1.1583234028866622
320200000 1.0918648539455733
320300000 1.1414706027998824
320400000 1.2666055301510823
320500000 1.207763852986157
320600000 1.1898058240598373
320700000 1.1084801066256431
320800000 1.0696997894172668
320900000 1.0951392171553571
321000000 1.1086970016418298
321100000 1.145126712760826
321200000 1.092119515358438
321300000 1.1291596440228522
321400000 0.9834799619235695
321500000 0.9911342487598658
321600000 1.1042739538364112
321700000 1.113056028015

345300000 0.19384019023407498
345400000 0.13510753473792472
345500000 0.19889601701147355
345600000 0.1509568227609065
345700000 0.1659670680998266
345800000 0.14630614111080434
345900000 0.11772962247988251
346000000 0.15277611314898729
346100000 0.12379381364899213
346200000 0.15561250561426082
346300000 0.11590472452581922
346400000 0.14460192339785563
346500000 0.12226288290498985
346600000 0.14304089710143209
346700000 0.126053613505784
346800000 0.11829373752728767
346900000 0.12946799448749421
347000000 0.12480820229921738
347100000 0.13339530602202151
347200000 0.11154366381964088
347300000 0.12175892044874363
best so far: 0
type: [1, 3, 1, 41, 1] 123
cases of this type: 206763
347400000 0.06409318074907529
347500000 0.013257860957443716
best so far: 0
type: [1, 3, 41, 1, 1] 123
cases of this type: 5043
best so far: 0
type: [1, 41, 1, 1, 3] 123
cases of this type: 3321
best so far: 0
type: [1, 41, 1, 3, 1] 123
cases of this type: 1107
best so far: 0
type: [1, 41, 3, 1, 1] 123
c